<a href="https://colab.research.google.com/github/scudilio/FIAP/blob/main/RN_previsao_preco_autos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Problema de Négocio

Objetivo: previsão de preços de carros.

# 2.Importando os dados

In [ ]:
import pandas as pd #processamento de conjunto de dados

In [ ]:
df = pd.read_csv("/content/autos.csv", encoding = "ISO-8859-1", sep = ",")

In [ ]:
df.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


# 3. Tratamento e Análise de Dados

In [ ]:
#excluindo informações irrelevantes para prever o preco
cols_to_drop = ["dateCrawled", "dateCreated", "monthOfRegistration", "nrOfPictures", "postalCode", "lastSeen"]
df = df.drop(cols_to_drop, axis = 1)

In [ ]:
df.describe()

,price,yearOfRegistration,powerPS,kilometer
count,3.715280e+05,371528.000000,371528.000000,371528.000000
mean,1.729514e+04,2004.577997,115.549477,125618.688228
std,3.587954e+06,92.866598,192.139578,40112.337051
min,0.000000e+00,1000.000000,0.000000,5000.000000
25%,1.150000e+03,1999.000000,70.000000,125000.000000
50%,2.950000e+03,2003.000000,105.000000,150000.000000
75%,7.200000e+03,2008.000000,150.000000,150000.000000
max,2.147484e+09,9999.000000,20000.000000,150000.000000


## buscando quais informacoes são relevantes para o modelo (analisando a variabilidade das informacoes)

In [ ]:
#analisando a variabilidade da feature name
df['name'].value_counts()

,count
name,
Ford_Fiesta,657
BMW_318i,627
Opel_Corsa,622
Volkswagen_Golf_1.4,603
BMW_316i,523
...,...
Audi_A4_Avant_Klima_Gruene_Plakette_TÜV_&AU_NEU_XENON,1
Renault_clio_in_gold_450VB_!!,1
Fiat_Doblo_1.6_Multijet,1


In [ ]:
df.drop(['name'], axis = 1, inplace = True)


In [ ]:
df["vehicleType"].value_counts()

,count
vehicleType,
limousine,95894
kleinwagen,80023
kombi,67564
bus,30201
cabrio,22898
coupe,19015
suv,14707
andere,3357


In [ ]:
df.head()

,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,fuelType,brand,notRepairedDamage
0,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,benzin,volkswagen,NaN
1,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,diesel,audi,ja
2,privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,diesel,jeep,NaN
3,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,benzin,volkswagen,nein
4,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,diesel,skoda,nein


In [ ]:
df["seller"].value_counts() #pouca informacao de venda comercial deletar a feature

,count
seller,
privat,371525
gewerblich,3


In [ ]:
df['offerType'].value_counts() #deletar tipo de oferta

,count
offerType,
Angebot,371516
Gesuch,12


In [ ]:
cols = ["seller", "offerType"]
df = df.drop(cols, axis = 1)

## Dados inconsistente

In [ ]:
df.describe()

,price,yearOfRegistration,powerPS,kilometer
count,3.715280e+05,371528.000000,371528.000000,371528.000000
mean,1.729514e+04,2004.577997,115.549477,125618.688228
std,3.587954e+06,92.866598,192.139578,40112.337051
min,0.000000e+00,1000.000000,0.000000,5000.000000
25%,1.150000e+03,1999.000000,70.000000,125000.000000
50%,2.950000e+03,2003.000000,105.000000,150000.000000
75%,7.200000e+03,2008.000000,150.000000,150000.000000
max,2.147484e+09,9999.000000,20000.000000,150000.000000


In [ ]:
df.loc[df['price'] <= 10]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,fuelType,brand,notRepairedDamage
7,0,test,limousine,1980,manuell,50,andere,40000,benzin,volkswagen,nein
40,0,test,NaN,1990,NaN,0,corsa,150000,benzin,opel,NaN
60,1,control,suv,1994,manuell,286,NaN,150000,NaN,sonstige_autos,NaN
91,1,control,limousine,1995,manuell,113,e_klasse,150000,diesel,mercedes_benz,nein
115,0,test,NaN,2017,manuell,0,golf,5000,benzin,volkswagen,NaN
...,...,...,...,...,...,...,...,...,...,...,...
371356,0,control,NaN,2000,manuell,65,corsa,150000,NaN,opel,ja
371392,0,test,kleinwagen,2002,manuell,60,fiesta,150000,benzin,ford,NaN
371402,0,control,kleinwagen,1999,manuell,53,swift,150000,benzin,suzuki,NaN
371431,0,control,kleinwagen,1999,manuell,37,arosa,150000,benzin,seat,ja


In [ ]:
df.loc[df['price'] <= 10]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,fuelType,brand,notRepairedDamage
7,0,test,limousine,1980,manuell,50,andere,40000,benzin,volkswagen,nein
40,0,test,NaN,1990,NaN,0,corsa,150000,benzin,opel,NaN
60,1,control,suv,1994,manuell,286,NaN,150000,NaN,sonstige_autos,NaN
91,1,control,limousine,1995,manuell,113,e_klasse,150000,diesel,mercedes_benz,nein
115,0,test,NaN,2017,manuell,0,golf,5000,benzin,volkswagen,NaN
...,...,...,...,...,...,...,...,...,...,...,...
371356,0,control,NaN,2000,manuell,65,corsa,150000,NaN,opel,ja
371392,0,test,kleinwagen,2002,manuell,60,fiesta,150000,benzin,ford,NaN
371402,0,control,kleinwagen,1999,manuell,53,swift,150000,benzin,suzuki,NaN
371431,0,control,kleinwagen,1999,manuell,37,arosa,150000,benzin,seat,ja


In [ ]:
df.loc[df['price'] >= 350000]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,fuelType,brand,notRepairedDamage
1846,579000,control,coupe,1980,manuell,277,andere,20000,benzin,bmw,nein
10649,420000,control,coupe,2004,manuell,483,911,50000,benzin,porsche,nein
14663,11111111,control,coupe,2003,manuell,64,polo,150000,benzin,volkswagen,NaN
16889,1000000,control,kombi,1998,NaN,0,mondeo,150000,benzin,ford,ja
20143,1250000,test,coupe,2016,manuell,500,911,5000,benzin,porsche,nein
...,...,...,...,...,...,...,...,...,...,...,...
364171,3890000,test,coupe,2006,NaN,799,NaN,5000,NaN,sonstige_autos,nein
365461,599000,control,coupe,1980,manuell,377,andere,5000,benzin,bmw,nein
366653,99999999,control,cabrio,1996,manuell,192,3er,150000,NaN,bmw,NaN
366861,3895000,test,coupe,2006,NaN,799,NaN,5000,benzin,sonstige_autos,nein


In [ ]:
df = df[df.price<350000]

In [ ]:
df.describe()

,price,yearOfRegistration,powerPS,kilometer
count,371409.000000,371409.000000,371409.000000,371409.000000
mean,5723.791187,2004.561182,115.489256,125632.873732
std,8719.852069,91.938146,191.427445,40094.119028
min,0.000000,1000.000000,0.000000,5000.000000
25%,1150.000000,1999.000000,70.000000,125000.000000
50%,2950.000000,2003.000000,105.000000,150000.000000
75%,7200.000000,2008.000000,150.000000,150000.000000
max,349000.000000,9999.000000,20000.000000,150000.000000


In [ ]:
#outra maneira de tratar o preco <10

#import numpy as np
#percentil_1 = np.percentile(df["price"], 1)
#df["price"] = df["price"].apply(lambda x: percentil_1() if x < 10 else x)

In [ ]:
#devido aos pontos extremos vamos substituir os dados com preco <10 pela mediana
mediana_preco = df['price'].median
df["price"] = df["price"].apply(lambda x: mediana_preco() if x < 10 else x)

In [ ]:
df['price'].mean()

5819.36981871737

In [ ]:
df['price'].median()

2950.0

In [ ]:
df.describe()

,price,yearOfRegistration,powerPS,kilometer
count,371409.000000,371409.000000,371409.000000,371409.000000
mean,5819.369819,2004.561182,115.489256,125632.873732
std,8672.629793,91.938146,191.427445,40094.119028
min,10.000000,1000.000000,0.000000,5000.000000
25%,1300.000000,1999.000000,70.000000,125000.000000
50%,2950.000000,2003.000000,105.000000,150000.000000
75%,7200.000000,2008.000000,150.000000,150000.000000
max,349000.000000,9999.000000,20000.000000,150000.000000


# Identificando dados nulos

In [ ]:
nulos = df.isnull().sum()

nulos

,0
price,0
abtest,0
vehicleType,37836
yearOfRegistration,0
gearbox,20169
powerPS,0
model,20441
kilometer,0
fuelType,33346
brand,0


In [ ]:
df.isnull()

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,fuelType,brand,notRepairedDamage
0,False,False,True,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,True,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
371523,False,False,True,False,True,False,True,False,True,False,True
371524,False,False,False,False,False,False,False,False,False,False,False
371525,False,False,False,False,False,False,False,False,False,False,False
371526,False,False,False,False,False,False,False,False,False,False,True


In [ ]:
df.head(10)

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,fuelType,brand,notRepairedDamage
0,480.0,test,NaN,1993,manuell,0,golf,150000,benzin,volkswagen,NaN
1,18300.0,test,coupe,2011,manuell,190,NaN,125000,diesel,audi,ja
2,9800.0,test,suv,2004,automatik,163,grand,125000,diesel,jeep,NaN
3,1500.0,test,kleinwagen,2001,manuell,75,golf,150000,benzin,volkswagen,nein
4,3600.0,test,kleinwagen,2008,manuell,69,fabia,90000,diesel,skoda,nein
5,650.0,test,limousine,1995,manuell,102,3er,150000,benzin,bmw,ja
6,2200.0,test,cabrio,2004,manuell,109,2_reihe,150000,benzin,peugeot,nein
7,2950.0,test,limousine,1980,manuell,50,andere,40000,benzin,volkswagen,nein
8,14500.0,control,bus,2014,manuell,125,c_max,30000,benzin,ford,NaN
9,999.0,test,kleinwagen,1998,manuell,101,golf,150000,NaN,volkswagen,NaN


In [ ]:
df['vehicleType'].loc[df['price']<1400].value_counts()

,count
vehicleType,
kleinwagen,34975
limousine,21891
kombi,14828
bus,3747
coupe,3071
cabrio,1727
andere,961
suv,508


In [ ]:
df['vehicleType'].value_counts() #limousine é a moda, valor frequente

,count
vehicleType,
limousine,95881
kleinwagen,80018
kombi,67554
bus,30198
cabrio,22888
coupe,18976
suv,14705
andere,3353


In [ ]:
df['vehicleType'].mode()

,vehicleType
0,limousine


In [ ]:
df['vehicleType'].fillna('limousine', inplace = True)

In [ ]:
#df.loc[pd.isnull(df['gearbox'])].describe()

In [ ]:
df['gearbox'].value_counts()

,count
gearbox,
manuell,274149
automatik,77091


In [ ]:
df.groupby('gearbox')['price'].mean()

,price
gearbox,
automatik,10530.038824
manuell,4672.275379


In [ ]:
df['gearbox'].fillna('manuell', inplace = True)

In [ ]:
df['model'].value_counts().head(50) #modelo mais frequente moda

,count
model,
golf,30061
andere,26389
3er,20565
polo,13088
corsa,12573
astra,10829
passat,10304
a4,10255
c_klasse,8774


In [ ]:
df['fuelType'].value_counts() #benzin

,count
fuelType,
benzin,223791
diesel,107735
lpg,5377
cng,571
hybrid,278
andere,207
elektro,104


In [ ]:
df['notRepairedDamage'].value_counts()

,count
notRepairedDamage,
nein,263117
ja,36276


In [ ]:
valores = {'model':'golf',
           'fuelType': 'benzin',
           'notRepairedDamage':'nein'}
df = df.fillna(value = valores)

In [ ]:
df.isnull().sum()

,0
price,0
abtest,0
vehicleType,0
yearOfRegistration,0
gearbox,0
powerPS,0
model,0
kilometer,0
fuelType,0
brand,0


In [ ]:
df.drop('abtest', axis = 1, inplace = True)

In [ ]:
df.head()

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,fuelType,brand,notRepairedDamage
0,480.0,limousine,1993,manuell,0,golf,150000,benzin,volkswagen,nein
1,18300.0,coupe,2011,manuell,190,golf,125000,diesel,audi,ja
2,9800.0,suv,2004,automatik,163,grand,125000,diesel,jeep,nein
3,1500.0,kleinwagen,2001,manuell,75,golf,150000,benzin,volkswagen,nein
4,3600.0,kleinwagen,2008,manuell,69,fabia,90000,diesel,skoda,nein


In [ ]:
df.shape

(371409, 10)

In [ ]:
df.isnull().sum()

,0
price,0
vehicleType,0
yearOfRegistration,0
gearbox,0
powerPS,0
model,0
kilometer,0
fuelType,0
brand,0
notRepairedDamage,0


In [ ]:
df.shape

(371409, 10)

## Pré processamento

## Pré-processamento dos dados categóricos

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enconder = OneHotEncoder()
encoded_data = enconder.fit_transform(df[['vehicleType', 'gearbox',  'fuelType', 'brand', 'notRepairedDamage']])

In [ ]:
#converter o resultado em um dataframe

encoded_df = pd.DataFrame(encoded_data.toarray(), columns = enconder.get_feature_names_out())

In [ ]:
# Concatenar os DataFrames
df_final = pd.concat([df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

# Opcional: Remover as colunas categóricas originais
df_final.drop(['vehicleType', 'gearbox', 'model', 'fuelType', 'brand', 'notRepairedDamage'], axis=1, inplace=True)


In [ ]:
df_final.head()

,price,yearOfRegistration,powerPS,kilometer,vehicleType_andere,vehicleType_bus,vehicleType_cabrio,vehicleType_coupe,vehicleType_kleinwagen,vehicleType_kombi,...,brand_smart,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,notRepairedDamage_ja,notRepairedDamage_nein
0,480.0,1993,0,150000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,18300.0,2011,190,125000,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,9800.0,2004,163,125000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1500.0,2001,75,150000,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,3600.0,2008,69,90000,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
# Mostrar todas as linhas que têm pelo menos um valor nulo
nulos = df_final[df_final.isnull().any(axis=1)]
nulos.head()

,price,yearOfRegistration,powerPS,kilometer,vehicleType_andere,vehicleType_bus,vehicleType_cabrio,vehicleType_coupe,vehicleType_kleinwagen,vehicleType_kombi,...,brand_smart,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,notRepairedDamage_ja,notRepairedDamage_nein


In [ ]:
df_final.isnull().sum()

,0
price,0
yearOfRegistration,0
powerPS,0
kilometer,0
vehicleType_andere,0
...,...
brand_trabant,0
brand_volkswagen,0
brand_volvo,0
notRepairedDamage_ja,0


In [ ]:
df_final.head()

,price,yearOfRegistration,powerPS,kilometer,vehicleType_andere,vehicleType_bus,vehicleType_cabrio,vehicleType_coupe,vehicleType_kleinwagen,vehicleType_kombi,...,brand_smart,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,notRepairedDamage_ja,notRepairedDamage_nein
0,480.0,1993,0,150000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,18300.0,2011,190,125000,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,9800.0,2004,163,125000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1500.0,2001,75,150000,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,3600.0,2008,69,90000,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
encoded_df.isnull().sum()

,0
vehicleType_andere,0
vehicleType_bus,0
vehicleType_cabrio,0
vehicleType_coupe,0
vehicleType_kleinwagen,0
...,...
brand_trabant,0
brand_volkswagen,0
brand_volvo,0
notRepairedDamage_ja,0


In [ ]:
df.isnull().sum()

,0
price,0
vehicleType,0
yearOfRegistration,0
gearbox,0
powerPS,0
model,0
kilometer,0
fuelType,0
brand,0
notRepairedDamage,0


## Pré-processamento Features Númericas

In [ ]:
from datetime import datetime

In [ ]:
ano = datetime.now().year

In [ ]:
ano

2024

In [ ]:
df_final["idade_veiculo"] = ano - df_final["yearOfRegistration"]

In [ ]:
df_final.drop(["yearOfRegistration"], axis = 1, inplace = True)

In [ ]:
df.head(1)

,price,vehicleType,gearbox,powerPS,model,kilometer,fuelType,brand,notRepairedDamage,idade_veiculo
0,480.0,limousine,manuell,0,golf,150000,benzin,volkswagen,nein,31


In [ ]:
from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import MinMaxScaler

In [ ]:
padronizacao = StandardScaler()
#normalizacao = MinMaxScaler()

In [ ]:
# Assuming df is already prepared and contains 'powerPS' and 'kilometer' columns
columns_to_scale = ['powerPS', 'kilometer','idade_veiculo']

In [ ]:
# transformacao nas features numericas usando a padronizacao
scaled_values = padronizacao.fit_transform(df_final[columns_to_scale])
#scaled_values = normalizacao.fit_transform(df_final[columns_to_scale])

In [ ]:
scaled_values

array([[-0.60330644,  0.60774896,  0.12574973],
       [ 0.38923804, -0.01578472, -0.07003433],
       [ 0.24819225, -0.01578472,  0.00610392],
       ...,
       [-0.07046677,  0.60774896,  0.09311906],
       [-0.08091461,  0.60774896,  0.0278577 ],
       [ 1.06834743, -1.88638577, -0.09178812]])

In [ ]:
# Replace the original columns with the scaled values
df_final[columns_to_scale] = scaled_values

In [ ]:
df_final.head(5)

,price,yearOfRegistration,powerPS,kilometer,vehicleType_andere,vehicleType_bus,vehicleType_cabrio,vehicleType_coupe,vehicleType_kleinwagen,vehicleType_kombi,...,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,notRepairedDamage_ja,notRepairedDamage_nein,idade_veiculo
0,480.0,1993,-0.603306,0.607749,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.125750
1,18300.0,2011,0.389238,-0.015785,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.070034
2,9800.0,2004,0.248192,-0.015785,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.006104
3,1500.0,2001,-0.211513,0.607749,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.038735
4,3600.0,2008,-0.242856,-0.888732,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.037404


In [ ]:
## Validacao cruzada Hold Out

from sklearn.model_selection import train_test_split

In [ ]:
X = df_final.drop('price', axis = 1)
y = df_final['price']

In [ ]:
X_test, X_train, y_test, y_train = train_test_split(X, y, test_size = 0.2, random_state = 420)

In [ ]:
## Arquitetura da Rede Neural Multilayer Perceptron


from sklearn.neural_network import MLPRegressor

In [ ]:
X.shape

(371409, 313)

In [ ]:
(157+1)/2

79.0

In [ ]:

rna = MLPRegressor(hidden_layer_sizes= (157,79), #camadas
                   max_iter = 20000,  #epocas
                   tol = 0.0000001,  #tolerancia de parada  (o que vir primeiro tolerancia ou epoca)
                   learning_rate_init = 0.01, #taxa de aprendizagem inicial
                   solver = "adam",  #metodo de otimização
                   activation = "relu", #função de ativação
                   learning_rate = "constant", #taxa de aprendizagem
                   verbose = 3,) #traz informaçoes

In [ ]:
rna.fit(X_train,y_train)

Iteration 1, loss = 54136417.23932317
Iteration 2, loss = 53013591.85648952
Iteration 3, loss = 51903059.37946107
Iteration 4, loss = 50823459.34121675
Iteration 5, loss = 49823490.93434530
Iteration 6, loss = 48854437.28055920
Iteration 7, loss = 47915275.84780943
Iteration 8, loss = 46956280.78318911
Iteration 9, loss = 45864764.80699050
Iteration 10, loss = 44355469.67188973
Iteration 11, loss = 43156308.28229059
Iteration 12, loss = 42084827.11701924
Iteration 13, loss = 41092757.32824335
Iteration 14, loss = 40156346.65295655
Iteration 15, loss = 39261517.25844085
Iteration 16, loss = 38399175.21446535
Iteration 17, loss = 37574210.45316745
Iteration 18, loss = 36786844.21707568
Iteration 19, loss = 36024005.47998777
Iteration 20, loss = 35289802.42860414
Iteration 21, loss = 34587559.65870420
Iteration 22, loss = 33905369.15653662
Iteration 23, loss = 33246208.44330529
Iteration 24, loss = 32607132.74562082
Iteration 25, loss = 31993829.08604317
Iteration 26, loss = 31399447.7079

MLPRegressor(activation='logistic', hidden_layer_sizes=(157, 79),
             learning_rate_init=0.01, max_iter=20000, tol=1e-07, verbose=3)

In [ ]:


# Initialize the scaler
#scaler = StandardScaler()
normalizacao = MinMaxScaler()

# Fit and transform the selected columns
#scaled_values = scaler.fit_transform(df_final[columns_to_scale])
scaled_values = normalizacao.fit_transform(df_final[columns_to_scale])

# Replace the original columns with the scaled values
df_final[columns_to_scale] = scaled_values

In [ ]:
from datetime import datetime

currentYear = datetime.now().year

In [ ]:
currentYear

2024

In [ ]:
# Assuming df is your DataFrame with a 'year' column
##current_year = 2024
df_final['vehicle_age'] = currentYear - df_final['yearOfRegistration']

# Drop the original year column if not needed
df_final.drop(['yearOfRegistration'], axis=1, inplace=True)

In [ ]:
df_final.head()

,price,powerPS,kilometer,vehicleType_andere,vehicleType_bus,vehicleType_cabrio,vehicleType_coupe,vehicleType_kleinwagen,vehicleType_kombi,vehicleType_limousine,...,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,notRepairedDamage_ja,notRepairedDamage_nein,vehicle_age
0,480.0,0,150000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,31
1,18300.0,190,125000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,13
2,9800.0,163,125000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,20
3,1500.0,75,150000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,23
4,3600.0,69,90000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,16


In [ ]:
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Assuming df is already prepared and contains 'powerPS' and 'kilometer' columns
columns_to_scale = ['powerPS', 'kilometer','vehicle_age']

# Initialize the scaler
#scaler = StandardScaler()
normalizacao = MinMaxScaler()

# Fit and transform the selected columns
#scaled_values = scaler.fit_transform(df_final[columns_to_scale])
scaled_values = normalizacao.fit_transform(df_final[columns_to_scale])

# Replace the original columns with the scaled values
df_final[columns_to_scale] = scaled_values


In [ ]:
df_final.head()

,price,powerPS,kilometer,vehicleType_andere,vehicleType_bus,vehicleType_cabrio,vehicleType_coupe,vehicleType_kleinwagen,vehicleType_kombi,vehicleType_limousine,...,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,notRepairedDamage_ja,notRepairedDamage_nein,vehicle_age
0,480.0,0.00000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.889654
1,18300.0,0.00950,0.827586,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.887654
2,9800.0,0.00815,0.827586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.888432
3,1500.0,0.00375,1.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.888765
4,3600.0,0.00345,0.586207,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.887988


## 6. Validação do Modelo

In [ ]:
X = df_final.drop('price', axis = 1)
y = df_final['price']

In [ ]:
X.head()

,powerPS,kilometer,vehicleType_andere,vehicleType_bus,vehicleType_cabrio,vehicleType_coupe,vehicleType_kleinwagen,vehicleType_kombi,vehicleType_limousine,vehicleType_suv,...,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,notRepairedDamage_ja,notRepairedDamage_nein,vehicle_age
0,0.00000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.889654
1,0.00950,0.827586,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.887654
2,0.00815,0.827586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.888432
3,0.00375,1.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.888765
4,0.00345,0.586207,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.887988


## Validação do Modelo - Hold out

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 420)

In [ ]:
x_train.shape

(297127, 313)

In [ ]:
(313+1)/2

157.0

## Treinamento da rede neural

In [ ]:
(157+1)/2

79.0

In [ ]:
#arquitetura da rede
from sklearn.neural_network import MLPRegressor

rna = MLPRegressor(hidden_layer_sizes= (157), #camadas
                   max_iter = 20000,  #epocas
                   tol = 0.0000001,  #tolerancia de parada  (o que vir primeiro tolerancia ou epoca)
                   learning_rate_init = 0.01, #taxa de aprendizagem inicial
                   solver = "adam",  #metodo de otimização
                   activation = "relu", #função de ativação
                   learning_rate = "constant", #taxa de aprendizagem
                   verbose = 3,) #traz informaçoes

In [ ]:
#37634363.35136126
#37673550.04731806

In [ ]:
rna.fit(x_train, y_train)

Iteration 1, loss = 19642658.61199560
Iteration 2, loss = 16177487.67315470
Iteration 3, loss = 15448556.55740774
Iteration 4, loss = 15059224.86073128
Iteration 5, loss = 14740803.12427920
Iteration 6, loss = 14533161.63745310
Iteration 7, loss = 14354144.43937819


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPRegressor(hidden_layer_sizes=(157, 78), learning_rate_init=0.01,
             max_iter=20000, tol=1e-07, verbose=3)

In [ ]:
#Previssão do conjunto de teste
predito = rna.predict(x_test)

## Avaliando o modelo

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
mean_absolute_error(y_test, predito)

2521.5347454303

In [ ]:
mean_squared_error(y_test, predito)

29050856.811176766

In [ ]:
r2_score(y_test, predito)

0.614642327695171

## Ajustando os parametros

In [ ]:
from sklearn.model_selection import GridSearchCV

# Grade de parâmetros para busca
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (100, 100)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant', 'adaptive'],
}

# MLPClassifier
mlp = MLPRegressor(max_iter=100)

# Grid Search
grid_search = GridSearchCV(mlp, param_grid, n_jobs=-1, cv=5)
grid_search.fit(x_train, y_train)


KeyboardInterrupt: 

In [ ]:
# Configurar a grade de parâmetros
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (100, 100)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant', 'adaptive'],
}

# Criar o MLPRegressor
mlp = MLPRegressor(max_iter=1000)  # Ajuste o max_iter se necessário

# Configurar o Grid Search
grid_search = GridSearchCV(mlp, param_grid, n_jobs=-1, cv=5, scoring='neg_mean_squared_error')  # Usando erro quadrático médio negativo

# Executar o Grid Search
grid_search.fit(x_train, y_train)

# Mostrar os melhores parâmetros
print("Melhores parâmetros encontrados:")
print(grid_search.best_params_)

# Avaliar o modelo com os melhores parâmetros
best_model = grid_search.best_estimator_
print("Desempenho no conjunto de teste:")
print("Erro quadrático médio:", -best_model.score(X_test, y_test))  # Negativo porque usamos 'neg_mean_squared_error'
